In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()"><button>Toggle Code</button></a>
''')

<h1><center>Generative Modeling of Neuroimaging Data using Generative
Adversarial Networks</center></h1>
<br>
<p><center>Andrew Van</center></p>
<h6><center>Advisor: Nico Dosenbach</center></h6>
<script type="text/javascript">
$(window).load(function(){
    Reveal.configure({
        transition: 'fade' // none/fade/slide/convex/concave/zoom
    })
});    
</script>

<h3><center>Overview</center></h3>

- What are GANs?
- 